In [1]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import joblib

# Generate random data for student grades
np.random.seed(42)
num_students = 5000
courses = [f"500{i}" for i in range(8)] 
student_ids = [f"S{i+1:03d}" for i in range(num_students)]

# Random scores (0-100)
scores = np.random.randint(50, 100, size=(num_students, len(courses)))

# Map scores to grades
def map_score_to_grade(score):
    if score >= 80:
        return "A"
    elif score >= 70:
        return "B"
    elif score >= 60:
        return "C"
    elif score >= 50:
        return "D"
    else:
        return "F"

# Create a DataFrame
data = pd.DataFrame(scores, columns=courses, index=student_ids)

# Generate grades for each student
data["Grades"] = [list(map(map_score_to_grade, row)) for row in data[courses].values]

# Add Final Score and Final Grade (calculated based on course scores)
data["Final Score"] = np.mean(data[courses].values, axis=1).astype(int)  # Average of all course scores
data["Final Grade"] = data["Final Score"].apply(map_score_to_grade)

# Encode grades to numerical values
label_encoder = LabelEncoder()
flattened_grades = [grade for sublist in data["Grades"] for grade in sublist]
label_encoder.fit(flattened_grades)  # Fit on all possible grades

# Prepare data for ANN
X = data[courses].values  # Features: scores of all courses
y_score = data["Final Score"].values  # Target: final score
y_grade = label_encoder.transform(data["Final Grade"])  # Target: final grade

# Split data into training and testing sets
X_train, X_test, y_train_score, y_test_score, y_train_grade, y_test_grade = train_test_split(
    X, y_score, y_grade, test_size=0.2, random_state=42
)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train ANN model for score prediction (regression)
model_score = MLPRegressor(hidden_layer_sizes=(64, 32), max_iter=1000, random_state=42, solver='adam')
model_score.fit(X_train_scaled, y_train_score)

# Train ANN model for grade prediction (classification)
model_grade = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=1000, random_state=42, solver='adam')
model_grade.fit(X_train_scaled, y_train_grade)

# Save the models and label encoder
joblib.dump(model_score, "student_score_model.pkl")
joblib.dump(model_grade, "student_grade_model.pkl")
joblib.dump(label_encoder, "label_encoder.pkl")
joblib.dump(scaler, "scaler.pkl")

print("Models and label encoder saved successfully!")

Models and label encoder saved successfully!
